In [53]:
import os
import river
from river import ensemble, evaluate, forest, metrics, neighbors, stream, tree

In [118]:
bases['chess']['converter'].keys()

KeyError: 'converter'

In [119]:
list(bases['chess']['fieldnames'])

['y', 'x']

In [126]:
import csv
import itertools
import os
import pandas as pd
import river
from river import ensemble, evaluate, forest, metrics, neighbors, stream, tree
from datetime import datetime

algos = {
    'tree': tree.ExtremelyFastDecisionTreeClassifier(),
    'forest': forest.ARFClassifier(),
    # 'ensemble': ensemble.LeveragingBaggingClassifier(),
    'neighbors': neighbors.SAMKNNClassifier()
}

bases = {
    'chess': {
        'data': 'datasets/artificial/chess/transientChessboard.data',
        'labels': 'datasets/artificial/chess/transientChessboard.labels',
        'fieldnames': {'x', 'y'},
        'drop': None},
    'rbf': {
        'data': 'datasets/artificial/rbf/movingRBF.data',
        'labels': 'datasets/artificial/rbf/movingRBF.labels',
        'fieldnames': {'x0', 'x1', 'x2', 'x3', 'x4',
                       'x5', 'x6', 'x7', 'x8', 'x9'},
        'drop': None},
    'sea': {
        'data': 'datasets/artificial/sea/SEA_training_data.csv',
        'labels': 'datasets/artificial/sea/SEA_training_class.csv',
        'fieldnames': {'x', 'y', 'z'},
        'drop': ['z']},
    'squares': {
        'data': 'datasets/artificial/movingSquares/movingSquares.data',
        'labels': 'datasets/artificial/movingSquares/movingSquares.labels',
        'fieldnames': {'x', 'y'},
        'drop': None},
}

all_metrics = [
    'Accuracy',
    'Precision',
    'Recall',
    'F1',
    'GeometricMean',
]

_metric = []
for m in all_metrics:
    _m = getattr(metrics, m)()
    _metric.append(_m)

for bas in bases:

    print(f'starting {bas}, at {datetime.now()}')

    try:
        f = open(bases[bas]['labels'], 'r')
        y = []
        for line in f.readlines():
            y.append(int(line))
        f.close()
    except:
        print('Y not found')
    try:
        f = open(bases[bas]['data'], 'r')
        x = csv.DictReader(
            f, delimiter=' ', quoting=csv.QUOTE_NONNUMERIC, fieldnames=bases[bas]['fieldnames'])
    except:
        print('X not found')
    
    for mod in algos:

        model = algos[mod]

        print(f'evaluating {mod}, at {datetime.now()}')

        try:
            w = open(f'results/{mod}_{bas}.csv', 'w')
            writer = csv.writer(w)
            writer.writerow(['index',
                             'Accuracy',
                             'Precision',
                             'Recall',
                             'F1',
                             'GeometricMean'
                             ])
        except:
            print('Result file can\'t be opened')

        for i, _x in enumerate(x):
            # Test Then Train
            # Test
            y_pred = model.predict_one(_x)
            # Train
            model.learn_one(_x, y[i])

            if y_pred is not None:
                a = [i]
                for m in _metric:
                    m.update(y[i], y_pred)
                    a.append(m.get())

                writer.writerow(a)
        f.close()

starting chess, at 2023-07-05 14:52:48.552221
evaluating tree, at 2023-07-05 14:52:48.779903
